In [ ]:
from dotenv import load_dotenv
import os
# load the environment variables
load_dotenv()

**NOTE** Python uses .rst format for sphinix documentation

### Testing OpenAI client 

In [ ]:
# importing openai client from langchain_openai
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# print(OpenAI.__doc__)

# need to pass 

# base_url --> OPENAI_API_BASE
# api_key  --> OPENAI_API_KEY



llm = ChatOpenAI(base_url=os.getenv("OPENAI_API_BASE"), api_key=os.getenv("OPENAI_API_KEY"))
llm.invoke("Who are you?").__dict__

In [ ]:
# Human Message Object
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

SystemPrompt = SystemMessage("You are an Expert Python Developer. " \
"Your Task is to write UnitTestCases for given Python Code using **PyTest** Framework. Do not add Extra Commentary")

code_input = """

```python
def add(x,y):
    return x + y
```

"""

messages = [SystemPrompt, code_input]
response = llm.invoke(messages)

response.__dict__



In [ ]:
print(response.content)

In [ ]:
response.usage_metadata

### Testing Azure Client

In [ ]:
from langchain_openai import AzureChatOpenAI

azure_client = AzureChatOpenAI(api_version=os.getenv("AZURE_OPENAI_VERSION"), azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"))

In [ ]:
response = azure_client.invoke("Generate a random json for user schema and only use 5-6 fields")

In [ ]:
response

### Structured Output

In [ ]:
from pydantic import BaseModel, Field
class ResponseFormatter(BaseModel):
    """Always use this tool to structure your response to the user."""
    answer: str = Field(description="The answer to the user's question")
    followup_question: str = Field(description="A followup question the user could ask")



structured_model = azure_client.bind_tools([ResponseFormatter])

In [ ]:
structured_output = structured_model.invoke("Who is the President of India?")

In [ ]:
structured_output

In [ ]:
structured_output.__dict__

### Testing OpenAI Client by changing the Base URL to Azure

In [ ]:
from langchain_openai import OpenAI

llm = OpenAI(base_url=os.getenv("AZURE_OPENAI_ENDPOINT"), api_key=os.getenv("AZURE_OPENAI_API_KEY"), model="gpt-4o-mini")

In [ ]:
llm.invoke("Hi")

### Testing Google AI

In [ ]:
# To run this code you need to install the following dependencies:
# pip install google-genai

import base64
import os
from google import genai
from google.genai import types


def generate():
    client = genai.Client(
        api_key=os.environ.get("GEMINI_API_KEY"),
    )

    model = "gemini-2.5-flash-lite"
    contents = [
        types.Content(
            role="user",
            parts=[
                types.Part.from_text(text="""INSERT_INPUT_HERE"""),
            ],
        ),
    ]
    tools = [
        types.Tool(googleSearch=types.GoogleSearch(
        )),
    ]
    generate_content_config = types.GenerateContentConfig(
        thinking_config = types.ThinkingConfig(
            thinking_budget=0,
        ),
        tools=tools,
    )

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        print(chunk.text, end="")

if __name__ == "__main__":
    generate()

### Testing Langchain Google GenAI

In [ ]:
# Getting Environment Variables

from pydantic import SecretStr

import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API Key: ")

print(os.environ["GOOGLE_API_KEY"])

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from utils.utils import load_config

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite")

config = load_config()

system_message = SystemMessage(content=config["Prompts"]["system message"])
messages = []
messages.append(system_message)

def chat_with_google(query):
    messages.append(HumanMessage(content=query))

    response = llm.invoke(messages)
    print(response.content, flush=True, end="")
    messages.append(AIMessage(content=response.content))


while True:

    query = input("write your code here(Enter 'exit' to terminate the chat): ")

    if query.lower() == "exit":
        print("😀 thanks for chatting 🚀. Feel free to connect again❗")
        break
    chat_with_google(query)


In [ ]:
from typing_extensions import TypedDict

In [ ]:
from typing import Required


class User(TypedDict):
    name: str
    username: str
    email: Required[str]

In [ ]:
user1 : User = {"name":"Ganesh", "username":"Ganesha@123", "password":124}

In [ ]:
user1

In [ ]:
import langgraph

In [ ]:
class State(TypedDict):
    graph_state: str

In [ ]:
# node 1 is conditional node 
def node_1(state):
    print("---Node 1---")
    return {"graph_state": state["graph_state"] + " I am"}



def node_2(state):
    print("---Node 2---")
    return {"graph_state": state["graph_state"] + " happy!"}

def node_3(state):
    print("---Node 3---")
    return {"graph_state": state["graph_state"] + " sad!"}

In [ ]:
import random
from typing import Literal


def decide_mood(state) -> Literal["node_2", "node_3"]:

    user_input = state["graph_state"]

    
    # Here, Let's do a 50/50 split between node 2, 3
    if random.random() < 0.5:
        return "node_2"
    
    return "node_3"

In [ ]:
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END


# build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)


# logic
builder.add_edge(START, "node_1")
builder.add_conditional_edges("node_1", decide_mood)
builder.add_edge("node_2", END)
builder.add_edge("node_3", END)


# Add
graph = builder.compile()
print(graph.__dict__)


In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
output = graph.invoke(State(graph_state="Hi, this is Ganesh"))

In [ ]:
builder.__dict__

In [ ]:
from typing import Literal, List
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display


class State(TypedDict):
    input: List[int]
    start: int
    end:   int
    mid:   int
    target: int


state1 = State()


def node_1(state):
    print("---node_1---")
    inputList = list(map(int, input("Enter a sorted List:").split()))
    state["input"] = inputList
    state["start"] = 0
    state["end"] = len(inputList) - 1
    print("End: ", state["end"])
    state["mid"] = int(0 + (state["end"] - state["start"]) // 2 )
    state1 = state
    
    return state1


def node_2(state):
    state = state
    print("---node_2---")
    target = int(input("Enter Target: "))
    state["target"] = target
    state1 = state
    
    return state1


def node_3(state):
    state = state
    print("---node_3---")
    state1 = state
    
    return state1


def node_4(state):
    state = state
    print("---node_4---")
    print(f"Target {state['target']} found at index {state['mid']}")
    state1 = state
    
    return state1


def node_5(state):
    state
    print("---node_5---")
    state["start"] = state["mid"] + 1
    print("Start:", state["start"])
    state1 = state
    
    return state1

def node_6(state):
    state = state
    print("---node_6---")
    state["end"] = state["mid"] - 1
    print("End:", state["end"])
    state1 = state
    
    return state1

def node_7(state):
    
    print("---node_7---")
    print(f"Element {state['target']} not found in arr {state['input']}")
    state1 = state
    
    return state1



def BinarySearch(state) -> Literal["node_4", "node_5", "node_6", "node_7"]:

    state["mid"] = state["start"] + (state["end"] - state["start"]) // 2


    if state["start"] > state["end"]:
        return "node_7"

    if state["input"][state["mid"]] == state["target"]:
        return "node_4"
    
    if state["input"][state["mid"]] < state["target"]:
        # set start = mid + 1
        return "node_5"
    
    if state["input"][state["mid"]] > state["target"]:
        # set end = mid - 1
        return "node_6"
    



builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)


builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_node("node_4", node_4)
builder.add_node("node_5", node_5)
builder.add_node("node_6", node_6)
builder.add_node("node_7", node_7)
builder.add_conditional_edges("node_3", BinarySearch)
builder.add_edge("node_4", END)
builder.add_edge("node_5", "node_3")
builder.add_edge("node_6", "node_3")
builder.add_edge("node_7", END)


graph = builder.compile(debug=True)

In [ ]:
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
graph.invoke(state1, stream_mode="updates")

this is Empty Cell